# NHN AI EasyMaker SDK 사용 예제

이 노트북에서는 NHN AI EasyMaker SDK를 사용하여 모델 평가를 생성하는 방법을 보여줍니다.

In [1]:
# 필요한 변수 설정
region = "kr1"
appkey = "YOUR_APP_KEY"
user_access_key_id = "YOUR_ACCESS_KEY_ID"
secret_access_key = "YOUR_SECRET_ACCESS_KEY"

# 모델 평가로 생성된 배치 추론의 결과를 업로드할 경로를 설정합니다.
# NHN Cloud Object Storage에 AI EasyMaker 시스템 계정 권한 추가 필요 : https://docs.nhncloud.com/ko/Machine%20Learning/AI%20EasyMaker/ko/console-guide/#1-nhn-cloud-object-storage-ai-easymaker
regression_result_upload_uri = "obs://YOUR_OBS_URI"
classification_result_upload_uri = "obs://YOUR_OBS_URI"

## SDK 초기화

SDK를 사용하기 위해 초기화합니다.

In [3]:
# 필요한 라이브러리 임포트
import easymaker

# SDK 초기화
easymaker.init(
    region=region,
    appkey=appkey,
    access_token=easymaker.sample.get_access_token(user_access_key_id, secret_access_key),
)

## 모델 평가 생성

생성된 모델을 기반으로 모델 평가를 생성합니다.

[파라미터]

| 이름                                        | 타입      | 필수 여부 | 기본값   | 유효 범위                                                          | 설명                                                              |
|-------------------------------------------|---------|-------|-------|----------------------------------------------------------------|-----------------------------------------------------------------|
| model_evaluation_name                     | String  | 필수    | 없음    | 최대 50자                                                         | 모델 평가 이름                                                        |
| description                               | String  | 선택    | 없음    | 최대 255자                                                        | 모델 평가에 대한 설명                                                    |
| model_id                                  | String  | 필수    | 없음    | 최대 36자                                                         | 평가할 모델 ID                                                       |
| objective_code                            | String  | 필수    | 없음    | "CLASSIFICATION", "REGRESSION"                 | 평가 목표                                                           |
| class_names                               | String  | 선택    | 없음    | 1~5000                                                         | 분류 모델에서 결과로 가능한 class 목록(`,`로 구분된 문자열이나 숫자)                     |
| instance_type_name                             | String  | 필수    | 없음    | 없음                                                             | 인스턴스 타입 이름(CLI로 조회 가능)                                          |
| input_data_uri                            | String  | 필수    | 없음    | 최대 255자                                                        | 입력 데이터 파일 경로(NHN Cloud Object Storage 또는 NHN Cloud NAS)         |
| input_data_type_code                      | String  | 필수    | 없음    | "CSV", "JSONL" | 입력 데이터 타입                                                       |
| target_field_name                         | String  | 필수    | 없음    | 최대 255자                                                        | 정답(Ground truth) 레이블의 필드 이름                                     |
| boot_storage_size                         | Integer | 필수    | 없음    | 50~2040                                                        | 모델 평가를 실행할 인스턴스의 부트 스토리지 크기(단위: GB)                             |
| data_storage_size                         | Integer | 필수    | 없음    | 300~10000                                                      | 모델 평가에 필요한 데이터를 다운로드, 가공할 저장 공간 크기(단위: GB)                      |
| timeout_hours                             | Integer | 선택    | 720    | 1~720                                                          | 최대 모델 평가 시간(단위: 시간)                                             |
| batch_inference_instance_type_name             | String  | 필수    | 없음    | 없음                                                             | 인스턴스 타입 이름(CLI로 조회 가능)                                          |
| batch_inference_instance_count            | Integer | 필수    | 없음    | 1~10                                                           | 배치 추론에 사용할 인스턴스 수                                               |
| batch_inference_pod_count                 | Integer | 필수    | 없음    | 1~100                                                          | 분산 학습을 적용할 노드 수                                                 |
| batch_inference_output_upload_uri         | String  | 필수    | 없음    | 최대 255자                                                        | 배치 추론 결과 파일이 업로드될 경로(NHN Cloud Object Storage 또는 NHN Cloud NAS) |
| batch_inference_max_batch_size            | Integer | 필수    | 없음    | 1~1000                                                         | 동시에 처리되는 데이터 샘플의 수                                              |
| batch_inference_inference_timeout_seconds | Integer | 필수    | 없음    | 1~1200                                                         | 단일 추론 요청의 최대 허용 시간                                              |
| use_log                                   | Boolean | 선택    | False | True, False                                                    | Log & Crash Search 서비스에 로그를 남길지 여부                              |
| wait                                      | Boolean | 선택    | True  | True, False                                                    | True: 생성이 완료된 이후 반환, False: 생성 요청 후 즉시 반환                       |


In [ ]:
# 인스턴스 목록 조회
instance_type_list = easymaker.ModelEvaluation.get_instance_type_list()
for instance in instance_type_list:
    instance.print_info()
instance_type_name = instance_type_list[0].name

In [5]:
# 회귀 모델 생성
regression_model = easymaker.Model().create_by_model_upload_uri(
    model_type_code=easymaker.sample.regression_model.model_type,
    model_upload_uri=easymaker.sample.regression_model.model_uri,
    model_name="easymaker_regression_model_sample",
)

[AI EasyMaker] Model create request complete. model_id: y9cm0h9p0mdp
[AI EasyMaker] Model create status: CREATE_REQUESTED (0:00:00) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:00:10) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:00:20) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:02:10) Please wait...
[AI EasyMaker] Model create status: CREATE_IN_PROGRESS (0:02:20) Please wait...
[AI EasyMaker] Model create complete. model_id: y9cm0h9p0mdp


In [13]:
# 회귀 모델 평가 생성
regression_model_evaluation = easymaker.ModelEvaluation().create(
    model_evaluation_name="regression_model_evaluation",
    description="regression model evaluation sample (wine quality regression)",
    model_id=regression_model.model_id,
    objective_code="REGRESSION",
    class_names="",
    instance_type_name=instance_type_name,
    input_data_uri=easymaker.sample.regression_model.model_evaluation.input_data_uri,
    input_data_type_code=easymaker.sample.regression_model.model_evaluation.input_data_type,
    target_field_name=easymaker.sample.regression_model.model_evaluation.target_field_name,
    timeout_hours=1,
    batch_inference_instance_type_name=instance_type_name,
    batch_inference_instance_count=1,
    batch_inference_pod_count=1,
    batch_inference_output_upload_uri=regression_result_upload_uri,
    batch_inference_max_batch_size=100,
    batch_inference_inference_timeout_seconds=1200,
    use_log=False,
    wait=True,
)

[AI EasyMaker] Model evaluation create request complete. model_evaluation_id: l6nhaco80b6f
[AI EasyMaker] ModelEvaluation create status: CREATE_REQUESTED (0:00:00) Please wait...
[AI EasyMaker] ModelEvaluation create status: CREATE_IN_PROGRESS (0:00:10) Please wait...
[AI EasyMaker] ModelEvaluation create status: CREATE_IN_PROGRESS (0:00:20) Please wait...
[AI EasyMaker] ModelEvaluation create status: CREATE_IN_PROGRESS (0:06:20) Please wait...
[AI EasyMaker] ModelEvaluation create status: CREATE_IN_PROGRESS (0:06:30) Please wait...
[AI EasyMaker] ModelEvaluation create status: RUNNING (0:06:40) Please wait...
[AI EasyMaker] ModelEvaluation create status: RUNNING (0:06:50) Please wait...
[AI EasyMaker] ModelEvaluation create status: RUNNING (0:22:10) Please wait...
[AI EasyMaker] ModelEvaluation create status: RUNNING (0:22:20) Please wait...
[AI EasyMaker] ModelEvaluation create status: COMPLETE_IN_PROGRESS (0:22:30) Please wait...
[AI EasyMaker] ModelEvaluation create status: COMPLET

In [15]:
regression_model_evaluation.print_info()

{
    "description": "regression model evaluation sample (wine quality regression)",
    "appKey": "qV5VIb2S1hr33zTS",
    "createdDatetime": "2025-05-19T18:09:39+09:00",
    "modelEvaluationId": "l6nhaco80b6f",
    "modelEvaluationName": "regression_model_evaluation",
    "modelEvaluationStatusCode": "COMPLETE",
    "model": {
        "appKey": "qV5VIb2S1hr33zTS",
        "createdDatetime": "2025-05-19T16:34:49+09:00",
        "updatedDatetime": "2025-05-19T16:37:24+09:00",
        "modelId": "y9cm0h9p0mdp",
        "modelName": "easymaker_regression_model_sample",
        "modelStatusCode": "ACTIVE",
        "frameworkCode": "PYTORCH",
        "modelSize": 0,
        "modelUploadUri": "obs://kr1-api-object-storage.nhncloudservice.com/v1/AUTH_ed79fa143403492fbaf3ce31f0c03314/easymaker-sample/tabular-regression/model",
        "modelTypeCode": "PYTORCH"
    },
    "objectiveCode": "REGRESSION",
    "classNames": "",
    "predictionLabelFieldName": "",
    "predictionScoreFieldName": ""

In [ ]:
# 분류 모델 생성
classification_model = easymaker.Model().create_by_model_upload_uri(
    model_type_code=easymaker.sample.classification_model.model_type,
    model_upload_uri=easymaker.sample.classification_model.model_uri,
    model_name="easymaker_classification_model_sample",
)

In [ ]:
# 분류 모델 평가 생성
classification_model_evaluation = easymaker.ModelEvaluation().create(
    model_evaluation_name="classification_model_evaluation",
    description="classification model evaluation sample (iris)",
    model_id=classification_model.model_id,
    objective_code="CLASSIFICATION",
    class_names=easymaker.sample.classification_model.model_evaluation.class_names,
    instance_type_name=instance_type_name,
    input_data_uri=easymaker.sample.classification_model.model_evaluation.input_data_uri,
    input_data_type_code=easymaker.sample.classification_model.model_evaluation.input_data_type,
    target_field_name=easymaker.sample.classification_model.model_evaluation.target_field_name,
    timeout_hours=1,
    batch_inference_instance_type_name=instance_type_name,
    batch_inference_instance_count=1,
    batch_inference_pod_count=1,
    batch_inference_output_upload_uri=classification_result_upload_uri,
    batch_inference_max_batch_size=100,
    batch_inference_inference_timeout_seconds=1200,
    use_log=False,
    wait=True,
)

In [24]:
classification_model_evaluation.print_info()

{
    "description": "classification model evaluation sample (iris)",
    "appKey": "qV5VIb2S1hr33zTS",
    "createdDatetime": "2025-05-20T15:16:44+09:00",
    "modelEvaluationId": "gde682fafn6k",
    "modelEvaluationName": "classification_model_evaluation",
    "modelEvaluationStatusCode": "COMPLETE",
    "model": {
        "appKey": "qV5VIb2S1hr33zTS",
        "createdDatetime": "2025-05-19T18:55:25+09:00",
        "updatedDatetime": "2025-05-19T18:57:59+09:00",
        "modelId": "zpcye8zsv64q",
        "modelName": "easymaker_regression_model",
        "modelStatusCode": "ACTIVE",
        "frameworkCode": "TENSORFLOW",
        "modelSize": 0,
        "modelUploadUri": "obs://kr1-api-object-storage.nhncloudservice.com/v1/AUTH_ed79fa143403492fbaf3ce31f0c03314/easymaker-sample/iris-tensorflow-model",
        "modelTypeCode": "TENSORFLOW"
    },
    "objectiveCode": "CLASSIFICATION",
    "classNames": "setosa,versicolor,virginica",
    "predictionLabelFieldName": "",
    "predictionSco

## 모델 평가 결과

metrics 필드에 평가 결과가 저장되어 있습니다.

[회귀 모델 필드 정보]

| 이름       | 설명    |
|----------|-------|
| metrics.meanAbsoluteError | MAE   |
| metrics.meanAbsolutePercentageError | MAPE  |
| metrics.rsquared | R-squared  |
| metrics.rootMeanSquaredError | RMSE  |
| metrics.rootMeanSquaredLogError | RMSLE |

[분류 모델 필드 정보]

| 이름                         | 설명            |
|----------------------------|---------------|
| metrics.auPrc              | PR AUC        |
| metrics.auRoc              | ROC AUC       |
| metrics.logLoss            | 로그 손실         |
| metrics.confidenceMetrics.confidenceThreshold | 신뢰도 임곗값       |
| metrics.confidenceMetrics.recall | 재현율           |
| metrics.confidenceMetrics.precision | 정밀도           |
| metrics.confidenceMetrics.falsePositiveRate | 거짓 양성률        |
| metrics.confidenceMetrics.f1Score | 마이크로 평균 F1 점수 |
| metrics.confidenceMetrics.f1ScoreMacro | 매크로 평균 F1 점수  |
| metrics.confidenceMetrics.confusionMatrix | 혼동 행렬  |


## 모델 평가 조회

생성된 모델 평가를 조회합니다.

In [19]:
# 목록 조회
evaluation_list = easymaker.ModelEvaluation.get_list()
for evaluation in evaluation_list:
    evaluation.print_info()

{
    "description": "classification model evaluation sample (iris)",
    "appKey": "qV5VIb2S1hr33zTS",
    "createdDatetime": "2025-05-20T13:46:40+09:00",
    "modelEvaluationId": "gde682fafn6k",
    "modelEvaluationName": "classification_model_evaluation",
    "modelEvaluationStatusCode": "COMPLETE",
    "model": {
        "appKey": "qV5VIb2S1hr33zTS",
        "createdDatetime": "2025-05-19T18:55:25+09:00",
        "updatedDatetime": "2025-05-19T18:57:59+09:00",
        "modelId": "zpcye8zsv64q",
        "modelName": "easymaker_regression_model",
        "modelStatusCode": "ACTIVE",
        "frameworkCode": "TENSORFLOW",
        "modelSize": 0,
        "modelUploadUri": "obs://kr1-api-object-storage.nhncloudservice.com/v1/AUTH_ed79fa143403492fbaf3ce31f0c03314/easymaker-sample/iris-tensorflow-model",
        "modelTypeCode": "TENSORFLOW"
    },
    "objectiveCode": "CLASSIFICATION",
    "classNames": null,
    "predictionLabelFieldName": null,
    "predictionScoreFieldName": null,
   

In [18]:
# 단건 조회
regression_model_evaluation = easymaker.ModelEvaluation(model_evaluation_id=regression_model_evaluation.id)

## 모델 평가 삭제

생성된 모델 평가를 삭제합니다.

In [20]:
# 모델 평가 삭제
regression_model_evaluation.delete()
# id 정보로 삭제
easymaker.ModelEvaluation.delete_by_id(classification_model_evaluation.id)

[AI EasyMaker] ModelEvaluation deletion request completed. ModelEvaluation ID : l6nhaco80b6f
[AI EasyMaker] ModelEvaluation deletion request completed. ModelEvaluation ID : gde682fafn6k
